<a href="https://colab.research.google.com/github/marioTavFer/somePython/blob/main/var_svar_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregamento de bibliotecas

Os pacotes que usaremos nessa seção estão listados abaixo. Sempre que possível, colocaremos os pacotes utilizados logo no início de cada seção.^[É possível que algum dos pacotes listados tenha sido retirado sem prévio aviso do CRAN. Nesses casos, por favor, entre em contato com o nosso suporte.]

In [ ]:
from bcb import sgs  # Importa o módulo "sgs" do pacote "bcb" para interagir com o Banco Central do Brasil
import pandas as pd  # Importa o pacote "pandas" para análise e manipulação de dados
import numpy as np  # Importa o pacote "numpy" para computação numérica eficiente
import statsmodels.api as sm  # Importa o pacote "statsmodels" para análise estatística
from statsmodels.tsa.stattools import adfuller  # Importa a função "adfuller" do módulo "stattools" do pacote "statsmodels.tsa" para teste de estacionaridade
from statsmodels.tsa.api import VAR  # Importa a classe "VAR" do módulo "api" do pacote "statsmodels.tsa" para modelos de Vetores Autorregressivos (VAR)
from statsmodels.tsa.vector_ar.svar_model import SVAR  # Importa a classe "SVAR" do módulo "svar_model" do pacote "statsmodels.tsa.vector_ar" para modelos de Vetores Autorregressivos Estruturais (SVAR)
from plotnine import *  # Importa o pacote "plotnine" para visualização de dados com a gramática de gráficos do R
from matplotlib import pyplot as plt

# Estimação

Para ilustrar, vamos considerar um exemplo envolvendo algumas variáveis bastante conhecidas:

(i) inflação mensal medida pelo IPCA;
(ii) expectativas em $t$ para $t+1$ para a taxa de inflação mensal;
(iii) IC-br;
(iv) taxa de câmbio R\$/US\$;
(v) taxa Selic anualizada;
(vi) taxa de desemprego medida pela PNAD Contínua.
(vii) IBC-br


Os dados são importados abaixo.

In [ ]:
# Importa os dados
dados = pd.read_csv("https://aluno.analisemacro.com.br/download/50929/?tmstv=1687806232")

dados.set_index("data", inplace = True)

# Altera o tipo do índice para Datetime e frequência mensal
dados.index = pd.DatetimeIndex(dados.index, freq = "MS")

Um ponto importante que devemos nos atentar, para garantir a estabilidade do modelo, é a estacionariedade das séries. O gráfico abaixo deixa isso mais claro.

In [ ]:
# Converter os dados para o formato "long"
dados_long = dados.reset_index().melt(id_vars = "data",
                        var_name = "variaveis",
                        value_name = "valores")

# Criar o gráfico
(ggplot(dados_long)
 + geom_line(aes(x="data", y="valores", color="variaveis"))
 + facet_wrap("variaveis", scales="free_y")
 + scale_x_date(date_labels = "%Y", date_breaks = "4 year")
)

Acaso o leitor aplique o Teste ADF visto na seção anterior, verá que algumas de nossas séries não são estacionárias. Isso, como vimos, pode ser um problema para a estabilidade do nosso VAR.

In [ ]:
def adfuller_test(dataframe, signif=0.05, verbose=False):
    """Realiza o teste ADFuller para verificar a estacionaridade das colunas de um DataFrame e imprime o relatório"""
    for coluna in dataframe.columns:
        series = dataframe[coluna]
        r = adfuller(series, autolag='AIC')
        output = {'estatística_do_teste': round(r[0], 4), 'valor_p': round(r[1], 4), 'número_de_lags': round(r[2], 4), 'número_de_observações': r[3]}
        valor_p = output['valor_p']

        def ajustar(valor, comprimento=6):
            return str(valor).ljust(comprimento)

        # Imprimir resumo
        print(f'Teste de Dickey-Fuller Aumentado em "{coluna}"', "\n", '-' * 47)
        print(f'Hipótese Nula: Os dados possuem raiz unitária. Não estacionários.')
        print(f'Nível de Significância = {signif}')
        print(f'Estatística do Teste    = {output["estatística_do_teste"]}')
        print(f'Número de Lags Escolhidos = {output["número_de_lags"]}')

        for chave, valor in r[4].items():
            print(f'Valor Crítico {ajustar(chave)} = {round(valor, 3)}')

        if valor_p <= signif:
            print(f" => Valor-P = {valor_p}. Rejeitando a Hipótese Nula.")
            print(f" => A série é Estacionária.")
        else:
            print(f" => Valor-P = {valor_p}. Evidência fraca para rejeitar a Hipótese Nula.")
            print(f" => A série não é Estacionária.")
        print("\n")

# Realiza o teste para as séries do df
adfuller_test(dados)

Assim, de modo a contornar o problema, vamos simplesmente diferenciar as séries que consideramos não estacionárias. Isso é feito abaixo.

In [ ]:
# Diferencia a série
dados_diff = pd.merge(dados[['ipca']], dados.drop('ipca', axis = 1).diff(), left_index = True, right_index = True).dropna()

Vejamos agora o resultado visual das séries diferenciadas.

In [ ]:
# Converter os dados para o formato "long"
dados_diff_long = dados_diff.reset_index().melt(id_vars = "data",
                                  var_name = "variaveis",
                                  value_name = "valores")

# Criar o gráfico
(ggplot(dados_diff_long)
 + geom_line(aes(x = "data", y = "valores", color = "variaveis"))
 + facet_wrap("variaveis", scales = "free_y")
 + scale_x_date(date_labels = "%Y", date_breaks = "4 year")
)

Agora nossos dados estão prontos para o uso.

# Estimando o modelo VAR

Para estimar o VAR, podemos utilizar os módulos e funções presentes na biblioteca `statsmodels`, especificamente `statsmodels.tsa.api`.

# Determinação da ordem de defasagem

Uma vez de posse dos nossos dados, já devidamente tratados, um problema imediato é determinar a ordem $p$ de defasagem do nosso modelo.

No Python esses critérios estão implementados na função `VAR` de forma automática, isto é, a função estimará o modelo com a defasagem escolhida de acordo com o critério selecionado no parâmetro `ic` do método `fit` do `VAR` (padrão é `aic`).

Alternativamente, é possível escolher o número máximo de defasagens no método `fit` pelo parâmetro `maxlags`.

Abaixo, implementamos para as variáveis do nosso objeto de dados.

In [ ]:
# Ajustar o modelo VAR com os dados que contêm as dummies sazonais
var = VAR(endog = dados_diff).fit(maxlags = 1, trend = 'c')

# verifica a ordem de defasagem do modelo estimado
var.k_ar

De modo a escolher um modelo mais **parcimonioso**, vamos optar aqui por $maxlags = 1$ (que seria a ordem escolhida pelo critério aic).

Isso porque, digamos que estamos interessados em analisar a trajetória da inflação mensal. Como a mesma apresenta sazonalidade, é importante ressaltar isso em nosso modelo.

Abaixo, podemos ver o modelo estimado.

In [ ]:
var.summary()

## Diagnóstico

Uma vez estimado o nosso modelo VAR, a próxima etapa é verificar sua estabilidade. Para começar, verificamos se os módulos dos autovalores são menores do que a unidade. Isso é feito com o código abaixo.

In [ ]:
var.roots

Garantida essa condição, podemos proceder testes clássicos sobre os resíduos do modelo, como o teste de autocorrelação, normalidade e o de heterocedasticidade. O código abaixo ilustra essa análise.

In [ ]:
# Teste de autocorrelação
var.test_whiteness()

# Teste de normalidade
var.test_normality()

Observe que temos problema com os resultados dos testes de autocorrelação e normalidade. Isto é, estamos rejeitando a hipótese nula de ausência de autocorrelação e distribuição normal.

A seguir, podemos ver os gráficos dos resíduos de cada uma das equações estimadas através do comando abaixo.

In [ ]:
residuals = var.resid.reset_index()

# Cria gráfico dos resíduos
# Converter os dados para o formato "long"
residuals_long = residuals.melt(id_vars = "data",
                              var_name = "variaveis",
                              value_name = "valores")

# Criar o gráfico
(ggplot(residuals_long)
 + geom_line(aes(x = "data", y = "valores", color = "variaveis"))
 + facet_wrap("variaveis", scales = "free_y")
 + scale_x_date(date_labels = "%Y", date_breaks = "4 year")
)

Por fim, também é importante verificar a **estabilidade estrutural** do modelo, através da análise do processo de flutuação empírica (EPS, no inglês).

Esse tipo de diagnóstico permite a identificação de mudanças estruturais nas relações analisadas dentro do modelo.

In [ ]:
var.is_stable()

## Previsão

Uma vez que tenhamos construído o nosso modelo VAR, ele também pode ser utilizado para gerarmos previsões.

In [ ]:
lag_order = var.k_ar

var.forecast(dados.values[-lag_order:], 5)

In [ ]:
var.plot_forecast(10);

## Função Impulso-Resposta

Como vimos no início, a modelagem VAR procura identificar a relação dinâmica existente em um conjunto previamente definido de variáveis. Dentro desse tipo de abordagem, pode ser interessante para o pesquisador verificar o impacto de um **choque** ou **impulso em uma variável sobre as outras**.

In [ ]:
irf = var.irf(10)

irf.plot(orth = False);

## Decomposição de Variância

Outro tópico interessante para ver a relação entre duas ou mais variáveis é a decomposição de variância, isto é, o quanto da variância de uma variável do nosso conjunto é explicado pelas demais variáveis.

No `Python`, ela é implementada com o método `fevd`.

In [ ]:
fevd = var.fevd(5)

fevd.summary()

Observe que a maior parte da variância da inflação é explicada pela própria variável, seguida pelas expectativas. As demais variáveis do nosso modelo têm pouca ou nenhuma influência na variância da inflação.

In [ ]:
var.fevd(20).plot();

## SVAR

Para ilustrar a operacionalização de um SVAR na prática, vamos considerar as variáveis desemprego diferenciado, selic diferenciado e ipca. O código  abaixo seleciona as variáveis e estima um VAR(1).

In [ ]:
dados_svar = dados_diff[['ipca', 'desocupacao', 'selic']]

Podemos ver a matriz de covariância desse modelo na tabela abaixo.

In [ ]:
dados_svar.cov()

A matriz de covariância mostra que os valores fora da diagonal são diferentes de zero, implicando que $A \neq I$. Dessa forma, a especificação reduzida pode não ser a mais correta.

Assim, de modo a incluir efeitos contemporâneos, devemos impor uma diferente estrutura para a matriz $A$.

Assim, nossa matriz $A$ deveria ser algo como

\begin{align}
A = \begin{bmatrix}
1 & 0 & 0\\
\alpha & 1 & 0 \\
\alpha & \alpha & 1
\end{bmatrix}. \nonumber
\end{align}

Abaixo estimamos o modelo estrutural e obtemos os IRFs impondo a restrição acima.


In [ ]:
amat = np.asarray([[1, 0, 0],['E', 1, 0],['E', 'E', 1]])

svar = SVAR(dados_svar, svar_type = 'A', A = amat)

modelo_svar = svar.fit(maxlags = 1)

modelo_svar.irf().plot();